In [1]:
import json
import pandas as pd
import numpy as np
import logging
import datetime as dt

import utilities.densmore_v3 as dns

In [2]:
import logging
logging.basicConfig(level=logging.INFO, filemode='w', 
                    filename='../data/api_football_data/03_fixtures_basic_stats/fixtures_basic_stats.log',
                    format='%(asctime)s  |  %(levelname)s  |  %(message)s', 
                    datefmt='%a %b %d | %I:%M:%S %p')

## <span style="color:steelblue">Converting Basic Stats JSON to CSV 

<table width=100%>
<tr>
<td style="text-align:left" bgcolor="lightsteelblue">
</td>    
</tr>

Unfortunately, the Pandas json_normalize method doesn't work on this one... several lists break up the flattening, so I wrote custom code to extract the data I needed into a single table that I can easily join to the main fixtures data.

In [3]:
fixtures_df = pd.read_csv('../data/api_football_data/02_aggregated_info/rapi_fixtures_2016_2019.csv')

with open('../data/api_football_data/03_fixtures_basic_stats/rapi_fixtures_basic_stats_2016_2019.json') as f:
    basic_stats = json.load(f)

In [4]:
fixture_basic_stats = []    
    
for i in range(len(basic_stats)):
    
    stats = {}
    
    stats['fx_id'] = int(basic_stats[i]['parameters']['fixture'])
    logging.info(f"Fixture ID set: {stats['fx_id']}")
    
    fix_base_info = fixtures_df[fixtures_df['fx_id'] == stats['fx_id']].to_dict(orient='list')

    stats['tm_h_id'] = fix_base_info['tm_h_id'][0]
    stats['tm_h_name'] = fix_base_info['tm_h_name'][0]
    stats['tm_a_id'] = fix_base_info['tm_a_id'][0]
    stats['tm_a_name'] = fix_base_info['tm_a_name'][0]
    
    logging.info(f"Home ID set to: {stats['tm_h_id']}")
    logging.info(f"Home ID set from: {fix_base_info['tm_h_id'][0]}")
    
    for team in basic_stats[i]['response']:
        stat_dict = {}
        
        if team['team']['id'] == stats['tm_h_id']:
            for stat in team['statistics']:
                s = stat['type']
                v = stat['value']
                stat_dict[s] = v
                
            stat_dict = {'h_' + key.lower().replace(' ', '_'): val for key, val in stat_dict.items()}    

            
        if team['team']['id'] == stats['tm_a_id']:
            for stat in team['statistics']:
                s = stat['type']
                v = stat['value']
                stat_dict[s] = v
                

            stat_dict = {'a_' + key.lower().replace(' ', '_'): val for key, val in stat_dict.items()}
                

        stats.update(stat_dict) 
        
    fixture_basic_stats.append(stats)

basic_stats_df = pd.DataFrame(fixture_basic_stats)
basic_stats_df.to_csv('../data/api_football_data/03_fixtures_basic_stats/rapi_fixtures_basic_stats_2016_2019.csv', index=False)

#### **Removing unnecessary columns for ELO Model**

In [5]:
basic_stats_df.columns

Index(['fx_id', 'tm_h_id', 'tm_h_name', 'tm_a_id', 'tm_a_name',
       'h_shots_on_goal', 'h_shots_off_goal', 'h_total_shots',
       'h_blocked_shots', 'h_shots_insidebox', 'h_shots_outsidebox', 'h_fouls',
       'h_corner_kicks', 'h_offsides', 'h_ball_possession', 'h_yellow_cards',
       'h_red_cards', 'h_goalkeeper_saves', 'h_total_passes',
       'h_passes_accurate', 'h_passes_%', 'a_shots_on_goal',
       'a_shots_off_goal', 'a_total_shots', 'a_blocked_shots',
       'a_shots_insidebox', 'a_shots_outsidebox', 'a_fouls', 'a_corner_kicks',
       'a_offsides', 'a_ball_possession', 'a_yellow_cards', 'a_red_cards',
       'a_goalkeeper_saves', 'a_total_passes', 'a_passes_accurate',
       'a_passes_%'],
      dtype='object')

In [6]:
model_stats = basic_stats_df[['fx_id', 'tm_h_id', 'tm_h_name', 'tm_a_id', 'tm_a_name', 
                              'h_ball_possession', 'h_passes_%', 'a_ball_possession', 'a_passes_%']].copy()

In [7]:
model_stats = model_stats.rename(axis=1, mapper={'h_ball_possession':'h_ball_poss', 'a_ball_possession':'a_ball_poss',
                                  'h_passes_%':'h_pass_acc', 'a_passes_%':'a_pass_acc'})

In [8]:
model_stats.to_csv('../data/api_football_data/04_model_data/model_stats_2016_2019.csv', index=False)

### **Combining Basic Stats and Match Info Data**

#### **Reading in CSVs**

In [9]:
model_stats

,fx_id,tm_h_id,tm_h_name,tm_a_id,tm_a_name,h_ball_poss,h_pass_acc,a_ball_poss,a_pass_acc
0,148290,1601,Toronto FC,1595,Seattle Sounders,54%,76%,46%,68%
1,148291,1614,Montreal Impact,1601,Toronto FC,43%,73%,57%,78%
2,148292,1601,Toronto FC,1614,Montreal Impact,50%,71%,50%,72%
3,148293,1595,Seattle Sounders,1610,Colorado Rapids,52%,79%,48%,77%
4,148294,1610,Colorado Rapids,1595,Seattle Sounders,64%,77%,36%,67%
...,...,...,...,...,...,...,...,...,...
1571,250195,1608,Atlanta United FC,1599,Philadelphia Union,53%,85%,47%,84%
1572,250196,1616,Los Angeles FC,1605,Los Angeles Galaxy,42%,80%,58%,83%
1573,250197,1616,Los Angeles FC,1595,Seattle Sounders,68%,88%,32%,68%
1574,250198,1608,Atlanta United FC,1601,Toronto FC,59%,87%,41%,78%


In [10]:
fixtures_df

,fx_id,fx_ref,fx_tz,fx_date,fx_time,fx_per_fst,fx_per_sec,fx_ven_id,fx_ven_name,fx_ven_city,...,gl_h,gl_a,sc_ht_h,sc_ht_a,sc_ft_h,sc_ft_a,sc_et_h,sc_et_a,sc_pen_h,sc_pen_a
0,148290,"Alan Kelly, Ireland",UTC,2016-12-11T01:00:00+00:00,1481418000,1481418000,1481421600,NaN,BMO Field,Toronto,...,0,0,0,0,0,0,0.0,0.0,4.0,5.0
1,148291,"Juan Guzman, USA",UTC,2016-11-23T01:00:00+00:00,1479862800,1479862800,1479866400,NaN,Olympic Stadium,Montreal,...,3,2,2,0,3,2,NaN,NaN,NaN,NaN
2,148292,"Jair Marrufo, USA",UTC,2016-12-01T00:00:00+00:00,1480550400,1480550400,1480554000,NaN,BMO Field,Toronto,...,5,2,2,1,3,2,5.0,2.0,NaN,NaN
3,148293,"Chris Penso, USA",UTC,2016-11-23T03:00:00+00:00,1479870000,1479870000,1479873600,NaN,CenturyLink Field,Seattle,...,2,1,1,1,2,1,NaN,NaN,NaN,NaN
4,148294,"Ricardo Salazar, USA",UTC,2016-11-27T21:00:00+00:00,1480280400,1480280400,1480284000,NaN,Dick's Sporting Goods Park,Commerce City,...,0,1,0,0,0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571,250195,"Ismail Elfath, USA",UTC,2019-10-25T00:00:00+00:00,1571961600,1571961600,1571965200,NaN,Mercedes-Benz Stadium,Atlanta,...,2,0,1,0,2,0,NaN,NaN,NaN,NaN
1572,250196,"Kevin Stott, USA",UTC,2019-10-25T02:30:00+00:00,1571970600,1571970600,1571974200,NaN,Banc of California Stadium,Los Angeles,...,5,3,2,1,5,3,NaN,NaN,NaN,NaN
1573,250197,"Jair Antonio Maruffo, USA",UTC,2019-10-30T02:00:00+00:00,1572400800,1572400800,1572404400,NaN,Banc of California Stadium,Los Angeles,...,1,3,1,2,1,3,NaN,NaN,NaN,NaN
1574,250198,"Alan Kelly, Ireland",UTC,2019-10-31T00:00:00+00:00,1572480000,1572480000,1572483600,NaN,Mercedes-Benz Stadium,Atlanta,...,1,2,1,1,1,2,NaN,NaN,NaN,NaN


#### **Filtering out non-regular-season  Fixtures**

In [11]:
fixtures_df = fixtures_df[fixtures_df['lg_rnd'].str.contains('Regular Season')].copy()
fixtures_df

,fx_id,fx_ref,fx_tz,fx_date,fx_time,fx_per_fst,fx_per_sec,fx_ven_id,fx_ven_name,fx_ven_city,...,gl_h,gl_a,sc_ht_h,sc_ht_a,sc_ft_h,sc_ft_a,sc_et_h,sc_et_a,sc_pen_h,sc_pen_a
17,148307,"Allen Chapman, USA",UTC,2016-03-06T18:30:00+00:00,1457289000,1457289000,1457292600,NaN,Red Bull Arena,Harrison,...,0,2,0,0,0,2,NaN,NaN,NaN,NaN
18,148308,"Villarrea Armando, USA",UTC,2016-03-06T19:00:00+00:00,1457290800,1457290800,1457294400,NaN,Toyota Park,Bridgeview,...,3,4,1,3,3,4,NaN,NaN,NaN,NaN
19,148309,"Chris Penso, USA",UTC,2016-03-06T19:00:00+00:00,1457290800,1457290800,1457294400,NaN,Orlando Citrus Bowl Stadium,Orlando,...,2,2,0,1,2,2,NaN,NaN,NaN,NaN
20,148310,"Ted Unkel, USA",UTC,2016-03-06T20:00:00+00:00,1457294400,1457294400,1457298000,NaN,Avaya Stadium,San Jose,...,1,0,0,0,1,0,NaN,NaN,NaN,NaN
21,148311,"Jose Carlos Rivero, USA",UTC,2016-03-06T20:00:00+00:00,1457294400,1457294400,1457298000,NaN,BBVA Compass Stadium,Houston,...,3,3,1,1,3,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,128571,"Drew Fischer, Canada",UTC,2019-10-06T20:00:00+00:00,1570392000,1570392000,1570395600,NaN,Talen Energy Stadium,Chester,...,1,2,0,2,1,2,NaN,NaN,NaN,NaN
1559,128572,"Rubiel Vazquez, USA",UTC,2019-10-06T20:00:00+00:00,1570392000,1570392000,1570395600,NaN,Providence Park,Portland,...,3,1,1,1,3,1,NaN,NaN,NaN,NaN
1560,128573,"Kevin Stott, USA",UTC,2019-10-06T20:00:00+00:00,1570392000,1570392000,1570395600,NaN,CenturyLink Field,Seattle,...,1,0,1,0,1,0,NaN,NaN,NaN,NaN
1561,128574,"Villarrea Armando, USA",UTC,2019-10-06T20:00:00+00:00,1570392000,1570392000,1570395600,NaN,BMO Field,Toronto,...,1,0,0,0,1,0,NaN,NaN,NaN,NaN


#### **Filtering out unnecessary columns from `fixtures_df`**

In [12]:
fixtures_df.columns

Index(['fx_id', 'fx_ref', 'fx_tz', 'fx_date', 'fx_time', 'fx_per_fst',
       'fx_per_sec', 'fx_ven_id', 'fx_ven_name', 'fx_ven_city', 'fx_sts_long',
       'fx_sts_shrt', 'fx_sts_elps', 'lg_id', 'lg_name', 'lg_ctry', 'lg_logo',
       'lg_flag', 'lg_seas', 'lg_rnd', 'tm_h_id', 'tm_h_name', 'tm_h_logo',
       'tm_h_win', 'tm_a_id', 'tm_a_name', 'tm_a_logo', 'tm_a_win', 'gl_h',
       'gl_a', 'sc_ht_h', 'sc_ht_a', 'sc_ft_h', 'sc_ft_a', 'sc_et_h',
       'sc_et_a', 'sc_pen_h', 'sc_pen_a'],
      dtype='object')

In [13]:
fixtures_df = fixtures_df[['fx_id', 'fx_tz', 'fx_date', 'fx_time', 
                           'fx_ven_name', 'fx_ven_city', 
                           'fx_sts_elps', 'lg_seas', 'lg_rnd', 
                           'tm_h_id', 'tm_h_name', 'tm_h_win', 
                           'tm_a_id', 'tm_a_name', 'tm_a_win', 
                           'gl_h', 'gl_a']].copy()

#### **Game Number Column**

In [14]:
# fixtures_df['game_num'] = fixtures_df['lg_rnd'].map(lambda x: int(x[-2:])).copy()

In [15]:
# fixtures_df['lg_seas'].value_counts()

In [16]:
# fixtures_2016 = fixtures_df[fixtures_df['lg_seas'] == 2016].copy()
# fixtures_2017 = fixtures_df[fixtures_df['lg_seas'] == 2017].copy()
# fixtures_2018 = fixtures_df[fixtures_df['lg_seas'] == 2018].copy()
# fixtures_2019 = fixtures_df[fixtures_df['lg_seas'] == 2019].copy()

In [17]:
# len(fixtures_2016), len(fixtures_2017), len(fixtures_2018), len(fixtures_2019)

In [18]:
# fixtures_2016['game_num'].value_counts().sort_index()
# fixtures_2017['game_num'].value_counts().sort_index()
# fixtures_2018['game_num'].value_counts().sort_index()
# fixtures_2019['game_num'].value_counts().sort_index()

#### **Renaming Time Column and Unpacking it into separate Date and Time Columns**

In [19]:
fixtures_df.rename(columns={'fx_time':'fx_utc'}, inplace=True)

In [20]:
fixtures_df['fx_date'] = fixtures_df['fx_utc'].map(lambda x: dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
fixtures_df['fx_time'] = fixtures_df['fx_utc'].map(lambda x: dt.datetime.fromtimestamp(x).strftime('%H:%M:%S'))

In [21]:
fixtures_df.drop(axis=1, columns=['fx_utc'], inplace=True)

#### **Creating Results Column based on `tm_h_win` and `tm_a_win`**

The target columns: 'tm_h_win' and 'tm_a_win' each have nulls in them whenever the game ends in a draw.  
So I'll create a new column (`results`) to use as a target column in classification and drop the other two.

In [22]:
fixtures_df['result'] = fixtures_df['tm_h_win'].map(lambda x: 'home' if x==True else ( 'away' if x==False else 'draw'))

In [23]:
fixtures_df['result'].value_counts()

home    800
draw    372
away    340
Name: result, dtype: int64

In [24]:
fixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 17 to 1562
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fx_id        1512 non-null   int64 
 1   fx_tz        1512 non-null   object
 2   fx_date      1512 non-null   object
 3   fx_ven_name  1512 non-null   object
 4   fx_ven_city  1482 non-null   object
 5   fx_sts_elps  1512 non-null   int64 
 6   lg_seas      1512 non-null   int64 
 7   lg_rnd       1512 non-null   object
 8   tm_h_id      1512 non-null   int64 
 9   tm_h_name    1512 non-null   object
 10  tm_h_win     1140 non-null   object
 11  tm_a_id      1512 non-null   int64 
 12  tm_a_name    1512 non-null   object
 13  tm_a_win     1140 non-null   object
 14  gl_h         1512 non-null   int64 
 15  gl_a         1512 non-null   int64 
 16  fx_time      1512 non-null   object
 17  result       1512 non-null   object
dtypes: int64(7), object(11)
memory usage: 224.4+ KB


#### **Removing Unnecessary Columns again**

In [25]:
fixtures_df.columns

Index(['fx_id', 'fx_tz', 'fx_date', 'fx_ven_name', 'fx_ven_city',
       'fx_sts_elps', 'lg_seas', 'lg_rnd', 'tm_h_id', 'tm_h_name', 'tm_h_win',
       'tm_a_id', 'tm_a_name', 'tm_a_win', 'gl_h', 'gl_a', 'fx_time',
       'result'],
      dtype='object')

In [26]:
fixtures_df = fixtures_df[['fx_id', 'fx_tz', 'fx_date', 'fx_time', 'lg_seas', 
                           'tm_h_id', 'tm_h_name', 'tm_a_id', 'tm_a_name', 
                           'gl_h', 'gl_a', 'result']].copy()

In [27]:
fixtures_df.head()
# fixtures_df.info()

,fx_id,fx_tz,fx_date,fx_time,lg_seas,tm_h_id,tm_h_name,tm_a_id,tm_a_name,gl_h,gl_a,result
17,148307,UTC,2016-03-06,10:30:00,2016,1602,New York Red Bulls,1601,Toronto FC,0,2,away
18,148308,UTC,2016-03-06,11:00:00,2016,1607,Chicago Fire,1604,New York City FC,3,4,away
19,148309,UTC,2016-03-06,11:00:00,2016,1598,Orlando City SC,1606,Real Salt Lake,2,2,draw
20,148310,UTC,2016-03-06,12:00:00,2016,1596,San Jose Earthquakes,1610,Colorado Rapids,1,0,home
21,148311,UTC,2016-03-06,12:00:00,2016,1600,Houston Dynamo,1609,New England Revolution,3,3,draw


#### **Merging the two dataframes on the fixture ID column**

In [28]:
df = pd.merge(left=fixtures_df, right=model_stats, how='left', on='fx_id')

In [29]:
df.columns

Index(['fx_id', 'fx_tz', 'fx_date', 'fx_time', 'lg_seas', 'tm_h_id_x',
       'tm_h_name_x', 'tm_a_id_x', 'tm_a_name_x', 'gl_h', 'gl_a', 'result',
       'tm_h_id_y', 'tm_h_name_y', 'tm_a_id_y', 'tm_a_name_y', 'h_ball_poss',
       'h_pass_acc', 'a_ball_poss', 'a_pass_acc'],
      dtype='object')

In [30]:
df['tm_h_id_x'].equals(df['tm_h_id_y']), df['tm_h_name_x'].equals(df['tm_h_name_y'])

(True, True)

In [31]:
df['tm_a_id_x'].equals(df['tm_a_id_y']), df['tm_a_name_x'].equals(df['tm_a_name_y'])

(True, True)

#### **Re-ordering Columns**

In [32]:
df = df[['fx_id', 'fx_tz', 'fx_date', 'fx_time', 'lg_seas', 
         'tm_h_id_x', 'tm_h_name_x', 'tm_a_id_x', 'tm_a_name_x', 'gl_h', 'gl_a', 'result', 
         'h_ball_poss', 'h_pass_acc', 'a_ball_poss', 'a_pass_acc']]

In [33]:
df.head()

,fx_id,fx_tz,fx_date,fx_time,lg_seas,tm_h_id_x,tm_h_name_x,tm_a_id_x,tm_a_name_x,gl_h,gl_a,result,h_ball_poss,h_pass_acc,a_ball_poss,a_pass_acc
0,148307,UTC,2016-03-06,10:30:00,2016,1602,New York Red Bulls,1601,Toronto FC,0,2,away,63%,72%,37%,58%
1,148308,UTC,2016-03-06,11:00:00,2016,1607,Chicago Fire,1604,New York City FC,3,4,away,39%,66%,61%,78%
2,148309,UTC,2016-03-06,11:00:00,2016,1598,Orlando City SC,1606,Real Salt Lake,2,2,draw,52%,80%,48%,76%
3,148310,UTC,2016-03-06,12:00:00,2016,1596,San Jose Earthquakes,1610,Colorado Rapids,1,0,home,41%,71%,59%,80%
4,148311,UTC,2016-03-06,12:00:00,2016,1600,Houston Dynamo,1609,New England Revolution,3,3,draw,52%,75%,48%,73%


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fx_id        1512 non-null   int64 
 1   fx_tz        1512 non-null   object
 2   fx_date      1512 non-null   object
 3   fx_time      1512 non-null   object
 4   lg_seas      1512 non-null   int64 
 5   tm_h_id_x    1512 non-null   int64 
 6   tm_h_name_x  1512 non-null   object
 7   tm_a_id_x    1512 non-null   int64 
 8   tm_a_name_x  1512 non-null   object
 9   gl_h         1512 non-null   int64 
 10  gl_a         1512 non-null   int64 
 11  result       1512 non-null   object
 12  h_ball_poss  1508 non-null   object
 13  h_pass_acc   1508 non-null   object
 14  a_ball_poss  1508 non-null   object
 15  a_pass_acc   1501 non-null   object
dtypes: int64(6), object(10)
memory usage: 200.8+ KB


In [35]:
df.rename(columns={'tm_h_id_x':'tm_h_id', 
                   'tm_a_id_x':'tm_a_id', 
                   'tm_h_name_x':'tm_h_name', 
                   'tm_a_name_x':'tm_a_name'}, inplace=True)

In [36]:
df['fx_id'] = df['fx_id'].map(lambda x: str(x))
df['tm_h_id'] = df['tm_h_id'].map(lambda x: str(x))
df['tm_a_id'] = df['tm_a_id'].map(lambda x: str(x))
df['tm_a_id'] = df['tm_a_id'].map(lambda x: str(x))

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fx_id        1512 non-null   object
 1   fx_tz        1512 non-null   object
 2   fx_date      1512 non-null   object
 3   fx_time      1512 non-null   object
 4   lg_seas      1512 non-null   int64 
 5   tm_h_id      1512 non-null   object
 6   tm_h_name    1512 non-null   object
 7   tm_a_id      1512 non-null   object
 8   tm_a_name    1512 non-null   object
 9   gl_h         1512 non-null   int64 
 10  gl_a         1512 non-null   int64 
 11  result       1512 non-null   object
 12  h_ball_poss  1508 non-null   object
 13  h_pass_acc   1508 non-null   object
 14  a_ball_poss  1508 non-null   object
 15  a_pass_acc   1501 non-null   object
dtypes: int64(3), object(13)
memory usage: 200.8+ KB


In [38]:
df['h_ball_poss'].fillna(value='50%', inplace=True)
df['a_ball_poss'].fillna(value='50%', inplace=True)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fx_id        1512 non-null   object
 1   fx_tz        1512 non-null   object
 2   fx_date      1512 non-null   object
 3   fx_time      1512 non-null   object
 4   lg_seas      1512 non-null   int64 
 5   tm_h_id      1512 non-null   object
 6   tm_h_name    1512 non-null   object
 7   tm_a_id      1512 non-null   object
 8   tm_a_name    1512 non-null   object
 9   gl_h         1512 non-null   int64 
 10  gl_a         1512 non-null   int64 
 11  result       1512 non-null   object
 12  h_ball_poss  1512 non-null   object
 13  h_pass_acc   1508 non-null   object
 14  a_ball_poss  1512 non-null   object
 15  a_pass_acc   1501 non-null   object
dtypes: int64(3), object(13)
memory usage: 200.8+ KB


In [40]:
# df[df.iloc[:, 15].isnull()]

#### **Converting Strings with % Symbol to Floats**

In [41]:
df = df.fillna(value=np.nan)

In [42]:
df['h_pass_acc'] = df['h_pass_acc'].map(lambda x: float(str(x).strip('%'))/100)
df['a_pass_acc'] = df['a_pass_acc'].map(lambda x: float(str(x).strip('%'))/100)

In [43]:
df['h_ball_poss'] = df['h_ball_poss'].map(lambda x: float(str(x).strip('%'))/100)
df['a_ball_poss'] = df['a_ball_poss'].map(lambda x: float(str(x).strip('%'))/100)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fx_id        1512 non-null   object 
 1   fx_tz        1512 non-null   object 
 2   fx_date      1512 non-null   object 
 3   fx_time      1512 non-null   object 
 4   lg_seas      1512 non-null   int64  
 5   tm_h_id      1512 non-null   object 
 6   tm_h_name    1512 non-null   object 
 7   tm_a_id      1512 non-null   object 
 8   tm_a_name    1512 non-null   object 
 9   gl_h         1512 non-null   int64  
 10  gl_a         1512 non-null   int64  
 11  result       1512 non-null   object 
 12  h_ball_poss  1512 non-null   float64
 13  h_pass_acc   1508 non-null   float64
 14  a_ball_poss  1512 non-null   float64
 15  a_pass_acc   1501 non-null   float64
dtypes: float64(4), int64(3), object(9)
memory usage: 200.8+ KB


In [45]:
h_pass_acc_mean = round(df['h_pass_acc'].mean(), 2)
a_pass_acc_mean = round(df['a_pass_acc'].mean(), 2)

h_pass_acc_mean, a_pass_acc_mean

(0.8, 0.78)

In [46]:
df['h_pass_acc'].fillna(value=h_pass_acc_mean, inplace=True)
df['a_pass_acc'].fillna(value=h_pass_acc_mean, inplace=True)

In [47]:
# df['h_pass_acc'].value_counts()
# df['a_pass_acc'].value_counts()

In [48]:
# df['a_pass_acc'].value_counts()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fx_id        1512 non-null   object 
 1   fx_tz        1512 non-null   object 
 2   fx_date      1512 non-null   object 
 3   fx_time      1512 non-null   object 
 4   lg_seas      1512 non-null   int64  
 5   tm_h_id      1512 non-null   object 
 6   tm_h_name    1512 non-null   object 
 7   tm_a_id      1512 non-null   object 
 8   tm_a_name    1512 non-null   object 
 9   gl_h         1512 non-null   int64  
 10  gl_a         1512 non-null   int64  
 11  result       1512 non-null   object 
 12  h_ball_poss  1512 non-null   float64
 13  h_pass_acc   1512 non-null   float64
 14  a_ball_poss  1512 non-null   float64
 15  a_pass_acc   1512 non-null   float64
dtypes: float64(4), int64(3), object(9)
memory usage: 200.8+ KB


In [50]:
df = df[['result', 'fx_id', 'lg_seas', 'fx_date', 
         'tm_h_name', 'tm_h_id', 'gl_h', 'h_ball_poss', 'h_pass_acc',  
         'tm_a_name', 'tm_a_id', 'gl_a', 'a_ball_poss', 'a_pass_acc']]

In [51]:
df = df.rename(axis=1, mapper={'fx_id':'game_id', 
                               'lg_seas':'season', 
                               'fx_date':'game_date', 
                               'tm_h_name':'h_name', 
                               'tm_h_id':'h_id', 
                               'gl_h':'h_goals', 
                               'h_ball_poss':'h_possession', 
                               'h_pass_acc':'h_pass_acc',  
                               'tm_a_name':'a_name', 
                               'tm_a_id':'a_id', 
                               'gl_a':'a_goals', 
                               'a_ball_poss':'a_possession', 
                               'a_pass_acc':'a_pass_acc'})

In [52]:
df.to_csv('../data/api_football_data/05_merged_data/merged_data_fullnames.csv', index=False)

In [53]:
teams_dict = {'Atlanta United FC' : 'atl',
              'Montreal Impact' : 'mtl',
              'Chicago Fire' : 'chi',
              'Columbus Crew' : 'crw',
              'DC United' : 'dcu',
              'FC Cincinnati' : 'cin',
              'New England Revolution' : 'ner',
              'New York City FC' : 'nyc',
              'New York Red Bulls' : 'nyrb',
              'Orlando City SC' : 'orl',
              'Philadelphia Union' : 'phi',
              'Toronto FC' : 'tfc',
              'Colorado Rapids' : 'cor',
              'FC Dallas' : 'fcd',
              'Houston Dynamo' : 'hou',
              'Los Angeles Galaxy' : 'lag',
              'Los Angeles FC' : 'lafc',
              'Minnesota United FC' : 'min',
              'Portland Timbers' : 'por',
              'Real Salt Lake' : 'rsl',
              'San Jose Earthquakes' : 'sje',
              'Seattle Sounders' : 'sea',
              'Sporting Kansas City' : 'skc',
              'Vancouver Whitecaps' : 'van'}

In [54]:
df['h_name'].replace(teams_dict, inplace=True)
df['a_name'].replace(teams_dict, inplace=True)

In [55]:
df.to_csv('../data/api_football_data/05_merged_data/merged_data_shortnames.csv', index=False)

In [56]:
df.columns

Index(['result', 'game_id', 'season', 'game_date', 'h_name', 'h_id', 'h_goals',
       'h_possession', 'h_pass_acc', 'a_name', 'a_id', 'a_goals',
       'a_possession', 'a_pass_acc'],
      dtype='object')

In [57]:
df[['h_name', 'h_id']].value_counts()

h_name  h_id
lag     1605    69
ner     1609    68
nyc     1604    68
tfc     1601    68
skc     1611    68
sje     1596    68
sea     1595    68
rsl     1606    68
por     1617    68
phi     1599    68
orl     1598    68
nyrb    1602    68
van     1603    68
chi     1607    68
mtl     1614    68
hou     1600    68
dcu     1615    68
crw     1613    68
cor     1610    68
fcd     1597    67
min     1612    51
atl     1608    51
lafc    1616    33
cin     2242    17
dtype: int64

In [58]:
counts_df = pd.DataFrame(df['h_id'].value_counts())

In [59]:
h_counts_df = pd.DataFrame(df[['h_name', 'h_id']].value_counts()).sort_values(by='h_name').reset_index()
a_counts_df = pd.DataFrame(df[['a_name', 'a_id']].value_counts()).sort_values(by='a_name').reset_index()

In [60]:
h_counts_df = h_counts_df.rename(axis=1, mapper={0:'h_count'})
a_counts_df = a_counts_df.rename(axis=1, mapper={0:'a_count'})

In [61]:
counts_df = pd.concat([h_counts_df, a_counts_df], axis=1)
counts_df['total_count'] = counts_df['h_count'] + counts_df['a_count']

In [62]:
counts_df = counts_df.sort_values(by='total_count', ascending=False).reset_index(drop=True)

In [63]:
counts_df

,h_name,h_id,h_count,a_name,a_id,a_count,total_count
0,ner,1609,68,ner,1609,68,136
1,chi,1607,68,chi,1607,68,136
2,tfc,1601,68,tfc,1601,68,136
3,skc,1611,68,skc,1611,68,136
4,sje,1596,68,sje,1596,68,136
5,sea,1595,68,sea,1595,68,136
6,rsl,1606,68,rsl,1606,68,136
7,por,1617,68,por,1617,68,136
8,phi,1599,68,phi,1599,68,136
9,orl,1598,68,orl,1598,68,136


In [64]:
counts_df.to_csv('../data/api_football_data/05_merged_data/game_counts.csv', index=False)